# 3.2 マルチラベルテキスト分類のファインチューニング

本 Notebook では、「2-1_multi_label_classification_train.ipynb」にてファインチューニングしたモデルの評価を行います。

## 事前準備

ライブラリのインストールやデータセットの準備などを行います。

### (1) ライブラリのインストール

利用するライブラリをインストールします。

In [ ]:
# ライブラリのインストール
!pip install scikit-learn==1.2.2 transformers==4.38.2 datasets==2.18.0 japanize_matplotlib==1.1.3 fsspec==2023.6.0 gcsfs==2023.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 41.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=3d162b454b35191d19ab7a33c5099b461d7efd903669dde31ab

In [ ]:
# 利用するライブラリのインポート
import os
import sys
import random

import japanize_matplotlib
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    f1_score,
    roc_auc_score,
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    Trainer
)
from transformers.trainer_utils import set_seed

In [ ]:
# 不要なwarningメッセージを除外
import warnings
warnings.simplefilter("ignore")

定数の定義を行います。




In [ ]:
# 定数の宣言
MAX_LENGTH = 512  # 入力の最大トークン
SEED_VALUE = 42  # 乱数SEED値の固定

# 乱数シード値をseed_valueに固定
def set_seed_function(seed_value: int = 0) -> None:
    """
    Set the seed value for reproducibility.

    Args:
        seed_value (int, optional): The seed value to set. Defaults to 0.
    """
    set_seed(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)


# 乱数シード値の固定
set_seed_function(SEED_VALUE)

以下のコードにて、保存済みのモデルを指定します。

モデルの保存先を変更した場合は、変数 `MODEL_INPUT` に変更後のパスを記入してください。変更していない場合は、そのままで問題ありません。

Google ドライブに保存する場合、「このノートブックに Google ドライブへのファイルへのアクセスを許可しますか？」という内容のポップアップが出現しますので、許可してください。



In [ ]:
# Google Colabで実行している場合、Googleドライブの保存先からモデルを読み込むる
if "google.colab" in sys.modules:
    # Googleドライブのマウント
    from google.colab import drive
    drive.mount('/content/drive')

    # モデルの保存先
    MODEL_INPUT = "drive/MyDrive/fine-turning_cookbook/chapter3/models/multi_label_classification"
else:
    # モデルの保存先
    MODEL_INPUT = "./models/multi_label_classification"

Mounted at /content/drive


### (2) データセットの準備

今回は、 Hugging Face で公開されているデータセットを利用します。

評価用データ (test) を利用します。

* URL: https://huggingface.co/datasets/go_emotions

In [ ]:
# データセットの読み込み
test_dataset = load_dataset('go_emotions', name='simplified', split="test")

# データセットの概要を表示する
print("テストデータ")
print(test_dataset)

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

テストデータ
Dataset({
    features: ['text', 'labels', 'id'],
    num_rows: 5427
})


In [ ]:
test_dataset = test_dataset.rename_columns({'labels': 'labels_old'})

In [ ]:
# ラベルの情報を保存
classes = test_dataset.features["labels_old"].feature.names  # ラベルの情報を取得
id2label = {}  # IDからラベルを取得するための辞書
label2id = {}  # ラベルからIDを取得するための辞書
for i, label in enumerate(test_dataset.features["labels_old"].feature.names):
    id2label[i] = label
    label2id[label] = i

## 評価

ファインチューニングしたモデルを評価します。
今回は、ファインチューニング時に利用した Trainer クラスを用いて推論・評価を行います。

そのため、 Data Collator や評価関数をファインチューニング時と同様に準備します。


### (1) ファインチューニング済みモデルとトークナイザの読み込み

トークナイザとモデルなどを準備します。

In [ ]:
# 利用するデバイスの確認 (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデルの読み込み
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_INPUT,
    ignore_mismatched_sizes=True,
    problem_type="multi_label_classification"
).to(device)

# tokenizerの読み込み
tokenizer = AutoTokenizer.from_pretrained(MODEL_INPUT)

In [ ]:
# Data collatorの設定
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# 評価関数の定義
def multi_label_metrics(predictions: np.ndarray, labels: np.ndarray, threshold: float = 0.5) -> dict:
    """
    マルチラベル分類の評価指標を計算する

    Args:
        predictions (numpy.ndarray): 各ラベルの予測確率
        labels (numpy.ndarray): 正解ラベル
        threshold (float, optional): 確率を2値に変換するためのしきい値.初期値は0.5.

    Returns:
        dict: midro-F1,macro-F1,AUC,正解率を含む評価指標の辞書
    """

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    y_true = labels
    accuracy = accuracy_score(y_true, y_pred)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average="micro")
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average="macro")
    roc_auc = roc_auc_score(y_true, y_pred, average="micro")

    return {"micro_f1": f1_micro_average, "macro_f1": f1_macro_average, "roc_auc": roc_auc, "accuracy": accuracy}


def compute_metrics(p: EvalPrediction) -> dict:
    """
    マルチラベル分類の評価指標を用意する

    Args:
        p (EvalPrediction): クラスのインスタンス

    Returns:
        dict: 計算された指標を含む辞書
    """
    predictions = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=predictions, labels=p.label_ids)
    return result


### (2) 推論

Trainer を用いて推論を実行します。

In [ ]:
trainer = Trainer(
    model=model,  # 利用するモデル
    tokenizer=tokenizer,  # トークナイザ
    data_collator=data_collator,  # データの前処理
    compute_metrics=compute_metrics,  # 評価指標の計算
)

In [ ]:
def tokenize_dataset(example) -> dict:
    """
    入力データをトークンIDに変換する

    Args:
        example (dict): 入力文とラベルを含む辞書(トークナイズ化対象のデータセット)

    Returns:
        dict: トークンIDに変換された入力文とラベルを含む辞書
    """

    all_labels = example["labels_old"]
    labels = [float(0.0) for _ in range(len(classes))]
    for label in all_labels:
        labels[label] = float(1.0)

    example_output = tokenizer(example["text"], truncation=True)
    example_output["labels"] = labels
    return example_output


# トークン化の実行。今回は1件ずつ処理したいため、batched=Trueを指定しない
tokenized_test_datasets = test_dataset.map(tokenize_dataset, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

### (3) 精度の評価

推論した結果を評価します。

まず、評価データに対する評価指標の値を確認します。

In [ ]:
trainer.evaluate(tokenized_test_datasets)

{'eval_loss': 0.08730247616767883,
 'eval_micro_f1': 0.5750942041988157,
 'eval_macro_f1': 0.43699883881729745,
 'eval_roc_auc': 0.7476648696564203,
 'eval_accuracy': 0.4580799705177815,
 'eval_runtime': 8.4747,
 'eval_samples_per_second': 640.377,
 'eval_steps_per_second': 80.121}

次に、`trainer.predict()` 関数を用いて推論結果を得ます。

In [ ]:
# trainer.evaluate()と同等の処理
predict = trainer.predict(tokenized_test_datasets)
test_eval = compute_metrics(predict)

# 評価指標の確認
print(test_eval)

{'micro_f1': 0.5750942041988157, 'macro_f1': 0.43699883881729745, 'roc_auc': 0.7476648696564203, 'accuracy': 0.4580799705177815}


推論結果の詳細を確認するために、変数 `predict` の結果を予測確率 (`probs`) と2値予測結果 (`y_pred`) に変換します。

In [ ]:
def multi_label_predict(predictions: np.ndarray, threshold: float = 0.5) -> np.ndarray:
    """
    しきい値を利用して,予測確率を2値予測に変換する.

    Args:
        predictions (numpy.ndarray): 各ラベルの予測確率
        threshold (float, optional): 確率を2値に変換するためのしきい値.初期値は0.5.

    Returns:
        Tuple[numpy.ndarray, numpy.ndarray]: 各ラベルの予測確率と2値予測結果
    """
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    return probs, y_pred

probs, y_pred = multi_label_predict(predict.predictions, threshold=0.5)


2値予測結果 (`y_pred`) を利用して、感情ラベルごとの正解率、 recall 、 precision を求めます。

In [ ]:
# ラベルごとの正解率、recall、precisionを計算
def multi_label_classification_report(y_true: list, y_pred: np.ndarray) -> pd.DataFrame:
    """
    マルチラベル分類の正解率、recall、precisionを計算する

    Args:
        y_true (list): 正解ラベル
        y_pred (numpy.ndarray): 予測ラベル

    Returns:
        pd.DataFrame: 分類結果
    """
    report = pd.DataFrame(
        columns=["precision", "recall", "f1-score"]
    )
    # y_trueをnumpy配列に変換
    y_true = np.array(y_true)
    # ラベルごとに正解率、recall、precisionを計算
    for i in range(len(classes)):
        precision, recall, f1_score, _ = precision_recall_fscore_support(y_true[:,i], y_pred[:,i], average="binary", zero_division=0)
        report.loc[model.config.id2label[i]] = [precision, recall, f1_score]

    return report


classification_report = multi_label_classification_report(tokenized_test_datasets["labels"], y_pred)
print(classification_report)


                precision    recall  f1-score
admiration       0.687739  0.712302  0.699805
amusement        0.770035  0.837121  0.802178
anger            0.496970  0.414141  0.451791
annoyance        0.467626  0.203125  0.283224
approval         0.538117  0.341880  0.418118
caring           0.466667  0.311111  0.373333
confusion        0.504587  0.359477  0.419847
curiosity        0.538760  0.489437  0.512915
desire           0.673913  0.373494  0.480620
disappointment   0.588235  0.132450  0.216216
disapproval      0.464865  0.322097  0.380531
disgust          0.611940  0.333333  0.431579
embarrassment    0.750000  0.162162  0.266667
excitement       0.571429  0.349515  0.433735
fear             0.689189  0.653846  0.671053
gratitude        0.934718  0.894886  0.914369
grief            0.000000  0.000000  0.000000
joy              0.643357  0.571429  0.605263
love             0.783673  0.806723  0.795031
nervousness      0.000000  0.000000  0.000000
optimism         0.642857  0.48387

レビューごとの推論結果の詳細を確認したい場合は、以下のコードのコメントアウトを外し、推論結果のcsvデータをご確認ください。

In [ ]:
# # 推論結果をcsvに保存
# classification_report.to_csv("results.csv", encoding="utf-8-sig")